## Loading data

In [2]:
import json
import requests
from rdflib import Graph
from requests.auth import HTTPDigestAuth

In [3]:
g = Graph()

In [4]:
g.parse("data/example-data.ttl", format="ttl")

<Graph identifier=Nce72321f9e92431b96e66f17728a50dd (<class 'rdflib.graph.Graph'>)>

## Use SPARQL Update

In [ ]:
# this works
from SPARQLWrapper import SPARQLWrapper
queryString = "INSERT DATA { GRAPH <https://golemlab.eu/data> { <https://golemlab.eu/data/1> a <https://golemlab.eu/data/corpus> . } }" 
sparql = SPARQLWrapper("http://localhost:8890/sparql-auth")
sparql.setHTTPAuth("DIGEST")
sparql.setCredentials("dba", "pwd123")
sparql.setQuery(queryString) 
sparql.method = 'POST' 
sparql.query()

documentation https://vos.openlinksw.com/owiki/wiki/VOS/VirtRDFInsert

## Upload a ttl file

this works:
```
curl --digest --user dba:pwd123 --url "http://localhost:8890/sparql-graph-crud-auth?graph=https://golemlab.eu/data" -X POST -T example-data.ttl
```

In [ ]:
ttl_content = g.serialize(format="ttl")

In [ ]:
graph_uri = "https://golemlab.eu/data"
url =  "http://localhost:8890/sparql-graph-crud-auth" + "?graph=" + graph_uri
print(url)
data = ttl_content.encode(encoding='utf-8')
response = requests.post(url=url, data=data, auth=HTTPDigestAuth("dba", "pwd123"), headers={'Content-Type': 'application/x-turtle'})
print(response.status_code)
print(response.text)

## Implemented in DB class

In [9]:
from sparql import DB
virtuoso = DB(triplestore="virtuoso", protocol="http",url="localhost",port="8890", username="dba", password="pwd123")

In [10]:
testdata = g.serialize()

In [11]:
virtuoso.crud_endpoint

'http://localhost:8890/sparql-graph-crud-auth'

In [13]:
virtuoso.upload(testdata, graph="https://golemlab.eu/data", format="ttl")

True